### Alte Patches, Bilder, Heatmaps und Results löschen

In [49]:
import os
import shutil

folders = ['test_picture', 'test_patches', 'heatmaps_output', 'final_results']

for folder in folders:
    if not os.path.exists(folder):
        print(f"Folder does not exist: {folder}")
        continue

    for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)  # Delete file
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)  # Delete folder
        except Exception as e:
            print(f"Failed to delete {file_path}. Reason: {e}")

### Bildvorverarbeitung

In [50]:
import cv2
import numpy as np
from pathlib import Path
from dataclasses import dataclass, field
import random
import shutil
import sys

@dataclass
class MultiScaleConfig:
    patch_size: int = 256
    
    # Skalierungsstufen (Teiler der Bildgröße)
    # Wir decken ein breites Spektrum ab.
    scale_divisors: list = field(default_factory=lambda: [1.5, 2.0, 2.5, 3.0, 4.0])
    
    overlap: float = 0.5 
    
    # --- NEUE LOGIK: Relative Mindestgröße ---
    # Wenn True, wird die min_window_size dynamisch pro Bild berechnet.
    use_relative_min_size: bool = True
    
    # Das kleinste Fenster muss mindestens X % der kürzeren Bildseite sein.
    # 0.12 = 12% (Bei 1080p sind das ca. 130px, bei 4K ca. 260px)
    min_relative_factor: float = 0.15
    
    # Fallback: Absolute Untergrenze in Pixeln, unter die wir NIEMALS gehen wollen,
    # selbst wenn das Bild winzig ist (um extremen Pixelmatsch zu vermeiden).
    absolute_pixel_floor: int = 128
    
    debug_view: bool = True

# --- Hilfsfunktionen ---

def get_square_patch(img, cx, cy, size, target_size=256):
    """Schneidet ein Quadrat aus und skaliert es auf target_size."""
    half = int(size // 2)
    x0, y0 = int(cx - half), int(cy - half)
    x1, y1 = int(x0 + size), int(y0 + size)
    h, w = img.shape[:2]
    
    pad_top = max(0, -y0); pad_bottom = max(0, y1 - h)
    pad_left = max(0, -x0); pad_right = max(0, x1 - w)
    
    if any([pad_top, pad_bottom, pad_left, pad_right]):
        img_padded = cv2.copyMakeBorder(img, pad_top, pad_bottom, pad_left, pad_right, cv2.BORDER_CONSTANT, value=[0,0,0])
        x0 += pad_left; x1 += pad_left; y0 += pad_top; y1 += pad_top
        patch = img_padded[y0:y1, x0:x1]
    else:
        patch = img[y0:y1, x0:x1]
        
    # Resize (LANCZOS4 ist am besten für Resizing, sowohl hoch als auch runter)
    if patch.shape[0] != target_size or patch.shape[1] != target_size:
        patch = cv2.resize(patch, (target_size, target_size), interpolation=cv2.INTER_LANCZOS4)
    return patch

def generate_multiscale_patches(img, cfg: MultiScaleConfig):
    """
    Erzeugt Patches mit dynamischer Mindestgröße basierend auf Bildauflösung.
    """
    h, w = img.shape[:2]
    base_size = min(h, w)
    
    # --- 1. Dynamische Berechnung der Mindestgröße ---
    if cfg.use_relative_min_size:
        # Berechne relative Größe (z.B. 12% von 1080p = 129px)
        dynamic_min = int(base_size * cfg.min_relative_factor)
        # Aber gehe niemals unter den absoluten Boden (z.B. 64px)
        limit_size = max(dynamic_min, cfg.absolute_pixel_floor)
    else:
        # Fester Wert, falls relative Logik deaktiviert ist (für Kompatibilität)
        limit_size = 256 # Default-Wert falls nicht anders definiert

    print(f"   Bildgröße: {w}x{h} -> Dyn. Min-Size: {limit_size}px")
    
    all_patches = []
    all_coords = []
    all_scale_ids = []

    for scale_idx, divisor in enumerate(cfg.scale_divisors):
        # Berechne Soll-Größe für diesen Teiler
        calc_size = int(base_size / divisor)
        
        # --- 2. Smart Clamping ---
        # Wenn die berechnete Größe kleiner ist als unser Limit:
        # Statt zu skippen, setzen wir die Größe auf das Limit.
        # Beispiel: Scale 6.0 ergibt 100px, Limit ist 130px -> Wir nehmen 130px.
        win_size = max(calc_size, limit_size)
        
        # Safety Check: Fenster darf nicht größer sein als das Bild selbst
        win_size = min(win_size, base_size)

        stride = max(1, int(win_size * (1 - cfg.overlap)))
        
        count_for_scale = 0
        for y in range(0, h - win_size + stride, stride):
            for x in range(0, w - win_size + stride, stride):
                cx_top = min(x, w - win_size)
                cy_top = min(y, h - win_size)
                
                center_x = cx_top + win_size // 2
                center_y = cy_top + win_size // 2
                
                patch = get_square_patch(img, center_x, center_y, win_size, cfg.patch_size)
                
                all_patches.append(patch)
                all_coords.append((cx_top, cy_top, win_size))
                all_scale_ids.append(scale_idx)
                count_for_scale += 1
        
        # Hinweis ausgeben, wenn Clamping passiert ist
        status = "✅"
        if calc_size < limit_size:
            status = "⚠️ (Clamped)" # Zeigt an, dass wir die Größe künstlich angehoben haben
            
        print(f"   {status} Scale {divisor} (Calc {calc_size}px -> Used {win_size}px): {count_for_scale} Patches.")

    return all_patches, all_coords, all_scale_ids

# --- Hauptfunktion ---

def main():
    # Pfade anpassen!
    input_dir = Path("~/DatenUbuntu/Studium/1. Semester/KI-Projekt/modeltest/pictures").expanduser()
    test_picture_dir = Path("test_picture") 
    output_root = Path("test_patches")    
    
    cfg = MultiScaleConfig()
    
    # Farben für Visualisierung
    colors = [
        (255, 0, 0),    # Riesig (Blau)
        (255, 255, 0),  # Cyan
        (0, 255, 0),    # Grün
        (0, 255, 255),  # Gelb
        (0, 165, 255),  # Orange
        (0, 0, 255)     # Klein (Rot)
    ]

    # Clean Init
    if test_picture_dir.exists(): shutil.rmtree(test_picture_dir)
    if output_root.exists(): shutil.rmtree(output_root)
    
    test_picture_dir.mkdir(parents=True, exist_ok=True)
    out_patches_root = output_root / "patches"
    out_patches_root.mkdir(parents=True, exist_ok=True)

    # Bilder finden
    if not input_dir.exists(): return
    img_files = list(input_dir.glob("*.[jJ][pP][gG]")) + list(input_dir.glob("*.[pP][nN][gG]"))
    if not img_files: return

    try:
        user_input = input(f"Wie viele Bilder aus '{len(img_files)}' zufällig wählen? (Zahl oder 'all'): ")
        num_to_process = len(img_files) if user_input.lower() == 'all' else int(user_input)
    except: num_to_process = 1

    selected_files = random.sample(img_files, min(num_to_process, len(img_files)))
    patch_metadata = []

    print(f"🚀 Starte Smart Multi-Scale Processing...")

    for img_file in selected_files:
        print(f"\nVerarbeite: {img_file.name}")
        shutil.copy2(img_file, test_picture_dir / img_file.name)
        img = cv2.imread(str(img_file))
        if img is None: continue
        
        h_orig, w_orig = img.shape[:2]
        vis_img = img.copy()
        img_patch_dir = out_patches_root / img_file.stem
        img_patch_dir.mkdir(parents=True, exist_ok=True)

        # B) Patches generieren
        patches, coords, scale_ids = generate_multiscale_patches(img, cfg)
        
        # Duplikate filtern? 
        # Optional: Hier könnte man noch exakte Duplikate (gleiche Koordinaten + Größe) rausfiltern,
        # falls durch das Clamping mehrere Scales auf die gleiche Größe fallen.
        # Der Einfachheit halber lassen wir es erstmal so, da unterschiedliche Scales oft 
        # leicht unterschiedliche Strides haben und somit nicht exakt deckungsgleich sind.

        for i, (p, (x, y, s), s_idx) in enumerate(zip(patches, coords, scale_ids)):
            name = f"S{s_idx}_p{i}.jpg"
            cv2.imwrite(str(img_patch_dir / name), p)
            patch_metadata.append(f"{img_file.name};{img_file.stem}/{name};{x};{y};{s};{w_orig};{h_orig}")
            
            color = colors[s_idx % len(colors)]
            thickness = max(1, int(s / 200))
            cv2.rectangle(vis_img, (x, y), (x+s, y+s), color, thickness)

        if cfg.debug_view:
            h, w = vis_img.shape[:2]
            sf = 800 / max(h, w)
            res_small = cv2.resize(vis_img, (int(w * sf), int(h * sf)))
            cv2.putText(res_small, f"MinSize: {int(min(h, w)*cfg.min_relative_factor)}px", (10, 30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            cv2.imshow("Smart Multi-Scale", res_small)
            if cv2.waitKey(100) & 0xFF == ord('q'): break

    with open(output_root / "metadata.txt", "w") as f:
        f.write("\n".join(patch_metadata))
    
    cv2.destroyAllWindows()
    print(f"✅ Fertig!")

if __name__ == "__main__":
    main()

🚀 Starte Smart Multi-Scale Processing...

Verarbeite: Passat_P3_frame3520.jpg
   Bildgröße: 720x1280 -> Dyn. Min-Size: 128px
   ✅ Scale 1.5 (Calc 480px -> Used 480px): 10 Patches.
   ✅ Scale 2.0 (Calc 360px -> Used 360px): 21 Patches.
   ✅ Scale 2.5 (Calc 288px -> Used 288px): 32 Patches.
   ✅ Scale 3.0 (Calc 240px -> Used 240px): 50 Patches.
   ✅ Scale 4.0 (Calc 180px -> Used 180px): 98 Patches.

Verarbeite: Passat_P3_frame460.jpg
   Bildgröße: 720x1280 -> Dyn. Min-Size: 128px
   ✅ Scale 1.5 (Calc 480px -> Used 480px): 10 Patches.
   ✅ Scale 2.0 (Calc 360px -> Used 360px): 21 Patches.
   ✅ Scale 2.5 (Calc 288px -> Used 288px): 32 Patches.
   ✅ Scale 3.0 (Calc 240px -> Used 240px): 50 Patches.
   ✅ Scale 4.0 (Calc 180px -> Used 180px): 98 Patches.

Verarbeite: Focus_P2_frame_1030.jpg
   Bildgröße: 720x1280 -> Dyn. Min-Size: 128px
   ✅ Scale 1.5 (Calc 480px -> Used 480px): 10 Patches.
   ✅ Scale 2.0 (Calc 360px -> Used 360px): 21 Patches.
   ✅ Scale 2.5 (Calc 288px -> Used 288px): 32 Pa

### Test Model

In [51]:
import os

# 1. System-Konfiguration (Unterdrückt Warnungen und behebt Berechtigungsprobleme)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
os.environ['MPLCONFIGDIR'] = os.path.join(os.getcwd(), "tmp_matplotlib_cache")
if not os.path.exists(os.environ['MPLCONFIGDIR']):
    os.makedirs(os.environ['MPLCONFIGDIR'], exist_ok=True)

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import load_img, img_to_array
from ipywidgets import interact, IntSlider

# 2. Pfade und Parameter (Basierend auf deiner Struktur)
model_path = 'final_model.keras'
folder_path = 'test_patches/patches'
img_size = (256, 256)

# 3. Modell laden
try:
    model = load_model(model_path)
    print(f"✅ Modell '{model_path}' erfolgreich geladen.")
except Exception as e:
    print(f"❌ Fehler beim Laden des Modells: {e}")
    model = None

# 4. Bilder-Liste erstellen
if os.path.exists(folder_path):
    files = sorted([str(p.relative_to(folder_path)) for p in Path(folder_path).rglob('*') 
                    if p.suffix.lower() in ('.png', '.jpg', '.jpeg', '.tif')])
    print(f"🔍 {len(files)} Bilder in Unterordnern von '{folder_path}' gefunden.")
else:
    print(f"❌ Ordner '{folder_path}' wurde nicht gefunden!")
    files = []

# 5. Anzeige-Funktion für den Slider
def browse_patches(index):
    if not files:
        print("Keine Bilder vorhanden.")
        return

    filename = files[index]
    img_path = os.path.join(folder_path, filename)
    
    # Bild laden (Grayscale + 256x256)
    img = load_img(img_path, target_size=img_size, color_mode='grayscale')
    img_array = img_to_array(img)
    
    img_tensor = np.expand_dims(img_array, axis=0)

    # Vorhersage
    prediction = model.predict(img_tensor, verbose=0)
    
    # Bestimmung der Wahrscheinlichkeit für Klasse 1
    if prediction.shape[-1] == 1:
        prob_1 = float(prediction[0][0])
    else:
        prob_1 = float(prediction[0][1])
    
    label = 1 if prob_1 > 0.5 else 0
    color = 'green' if label == 1 else 'red'

    # Visualisierung
    plt.figure(figsize=(6, 6))
    plt.imshow(img_array.squeeze(), cmap='gray')
    
    title_str = (f"Bild {index+1}/{len(files)}: {filename}\n"
                 f"KLASSE: {label} | Wahrsch. Klasse 1: {prob_1:.4f}")
    
    plt.title(title_str, color=color, fontsize=12, fontweight='bold')
    plt.axis('off')
    plt.show()

# 6. Interaktives Element starten
if files and model:
    print("Nutze den Slider oder die Pfeiltasten deiner Tastatur zum Durchblättern:")
    interact(browse_patches, index=IntSlider(
        min=0, 
        max=len(files)-1, 
        step=1, 
        value=0, 
        description='Patch-Index:',
        layout={'width': '500px'}
    ))

✅ Modell 'final_model.keras' erfolgreich geladen.
🔍 37734 Bilder in Unterordnern von 'test_patches/patches' gefunden.
Nutze den Slider oder die Pfeiltasten deiner Tastatur zum Durchblättern:


interactive(children=(IntSlider(value=0, description='Patch-Index:', layout=Layout(width='500px'), max=37733),…

In [52]:
import cv2
import numpy as np
import os
from pathlib import Path
import tensorflow as tf
import gc
from tensorflow.keras.models import load_model

# --- 1. GPU & System Setup (Robust) ---
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
# Wichtig: Erlaubt TensorFlow, Speicher dynamisch zu nehmen
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true' 
os.environ['MPLCONFIGDIR'] = str(Path.home() / ".matplotlib_cache")
Path(os.environ['MPLCONFIGDIR']).mkdir(exist_ok=True)

def setup_gpu():
    try:
        gpus = tf.config.list_physical_devices('GPU')
        if gpus:
            for gpu in gpus:
                try:
                    tf.config.experimental.set_memory_growth(gpu, True)
                except:
                    pass
            print(f"✅ GPU Beschleunigung aktiv.")
    except RuntimeError:
        print("ℹ️ GPU Setup übersprungen.")
setup_gpu()

# --- 2. Konfiguration ---
class QRFinalConfig:
    model_path = 'final_model.keras'
    base_path = Path('test_patches')
    patch_folder = base_path / "patches"
    metadata_file = base_path / "metadata.txt"
    original_img_dir = Path('test_picture')
    output_dir = Path('final_results')
    
    threshold = 0.65       
    start_batch_size = 32 

# --- 3. Die kombinierte Pipeline ---
def run_combined_pipeline():
    cfg = QRFinalConfig()
    cfg.output_dir.mkdir(exist_ok=True)
    
    if not cfg.metadata_file.exists():
        print("❌ Metadaten-Datei nicht gefunden!")
        return

    model = load_model(cfg.model_path, compile=False)
    
    with open(cfg.metadata_file, "r") as f:
        lines = [l.strip().split(";") for l in f.readlines()]

    images_dict = {}
    for img_name, rel_path, px, py, ps, w_orig, h_orig in lines:
        if img_name not in images_dict:
            images_dict[img_name] = {"w": int(w_orig), "h": int(h_orig), "patches": []}
        images_dict[img_name]["patches"].append({"path": rel_path, "x": int(px), "y": int(py), "s": int(ps)})

    print(f"--- Analyse startet (Versuche Batch-Size: {cfg.start_batch_size}) ---")

    for img_name, info in images_dict.items():
        orig_img = cv2.imread(str(cfg.original_img_dir / img_name))
        if orig_img is None: continue
        h_orig, w_orig = info["h"], info["w"]
        
        patch_list = info["patches"]
        if not patch_list: continue

        all_patch_imgs = []
        for p in patch_list:
            p_img = cv2.imread(str(cfg.patch_folder / p["path"]), cv2.IMREAD_GRAYSCALE)
            if p_img is None: continue
            p_img = cv2.resize(p_img, (256, 256), interpolation=cv2.INTER_LINEAR)
            all_patch_imgs.append(p_img.astype(np.float32))

        if not all_patch_imgs: continue

        # Daten vorbereiten
        input_batch = np.expand_dims(np.array(all_patch_imgs), axis=-1)

        preds = None
        current_bs = cfg.start_batch_size
        
        while current_bs >= 1:
            try:
                preds = model.predict(input_batch, batch_size=current_bs, verbose=0)
                break # Erfolg! Raus aus der Schleife
            except tf.errors.ResourceExhaustedError:
                current_bs //= 2
                gc.collect() # Speicher aufräumen
                if current_bs < 1:
                    print(f"❌ Bild {img_name} ist zu groß für den VRAM.")
                    preds = None
                    break
        
        if preds is None: continue

        # --- HEATMAP REKONSTRUKTION ---
        heatmap_max = np.zeros((h_orig, w_orig), dtype=np.float32)
        max_prob = 0.0
        
        for i, prob_vec in enumerate(preds):
            prob = float(prob_vec[0]) if len(prob_vec) == 1 else float(prob_vec[1])
            max_prob = max(max_prob, prob)
            
            p = patch_list[i]
            x, y, s = p["x"], p["y"], p["s"]
            heatmap_max[y:y+s, x:x+s] = np.maximum(heatmap_max[y:y+s, x:x+s], prob)

        # Speicher freigeben für nächstes Bild
        del input_batch, all_patch_imgs, preds
        gc.collect()

        _, thresh = cv2.threshold(heatmap_max, cfg.threshold, 1.0, cv2.THRESH_BINARY)
        thresh_8bit = (thresh * 255).astype(np.uint8)
        
        contours, _ = cv2.findContours(thresh_8bit, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        heatmap_8bit = (heatmap_max * 255).astype(np.uint8)
        heatmap_vis = cv2.GaussianBlur(heatmap_8bit, (15, 15), 0)
        heatmap_color = cv2.applyColorMap(heatmap_vis, cv2.COLORMAP_JET)
        overlay = cv2.addWeighted(orig_img, 0.6, heatmap_color, 0.4, 0)
        
        found_count = 0
        for cnt in contours:
            x, y, w, h = cv2.boundingRect(cnt)
            if w > 15 and h > 15:
                cv2.rectangle(overlay, (x, y), (x + w, y + h), (0, 255, 0), 4)
                found_count += 1

        status = "QR GEFUNDEN" if max_prob >= cfg.threshold else "KEIN TREFFER"
        cv2.rectangle(overlay, (0, 0), (overlay.shape[1], 60), (0, 0, 0), -1)
        
        # Info anzeigen (inklusive genutzter Batch Size, falls reduziert)
        bs_info = "" if current_bs == cfg.start_batch_size else f"(BS:{current_bs})"
        cv2.putText(overlay, f"{status} | Prob: {max_prob:.1%} {bs_info}", 
                    (20, 42), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 3)

        cv2.imwrite(str(cfg.output_dir / f"final_{img_name}"), overlay)
        print(f"✅ {img_name}: {status} ({max_prob:.2%}) {bs_info}")

    tf.keras.backend.clear_session()

if __name__ == "__main__":
    run_combined_pipeline()

✅ GPU Beschleunigung aktiv.
--- Analyse startet (Versuche Batch-Size: 32) ---
✅ Passat_P3_frame3520.jpg: QR GEFUNDEN (83.68%) 
✅ Passat_P3_frame460.jpg: QR GEFUNDEN (83.68%) 
✅ Focus_P2_frame_1030.jpg: QR GEFUNDEN (89.77%) 
✅ Focus_P1_frame_1430.jpg: QR GEFUNDEN (69.88%) 
✅ Passat_P3_frame380.jpg: QR GEFUNDEN (84.14%) 
✅ Seat_N_47.jpg: KEIN TREFFER (19.31%) 
✅ Passat_P1_frame570.jpg: QR GEFUNDEN (96.00%) 
✅ Passat_N3_frame2120.jpg: KEIN TREFFER (39.50%) 
✅ Passat_P3_frame2910.jpg: QR GEFUNDEN (93.12%) 
✅ CKlasse_Bilderreihe 5_P_IMG_9514_rot-3.jpg: QR GEFUNDEN (91.92%) 
✅ Focus_P1_frame_930.jpg: QR GEFUNDEN (83.80%) 
✅ Focus_P1_frame_3540.jpg: QR GEFUNDEN (95.44%) 
✅ Focus_P1_frame_730.jpg: QR GEFUNDEN (97.71%) 
✅ Focus_P1_frame_3220.jpg: QR GEFUNDEN (95.22%) 
✅ Passat_N1_frame3050.jpg: KEIN TREFFER (63.03%) 
✅ GKLASSE_NEU_P1_frame1980.jpg: QR GEFUNDEN (84.84%) 
✅ Passat_P2_frame3090.jpg: QR GEFUNDEN (87.88%) 
✅ Golf_IV_P_43.jpg: QR GEFUNDEN (99.26%) 
✅ IMG_6903.jpg: QR GEFUNDEN (82.92%

## Heatmap

In [53]:
import cv2
import numpy as np
import os
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.models import load_model

# --- 1. Robustes GPU-Setup ---
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

def setup_gpu():
    try:
        gpus = tf.config.list_physical_devices('GPU')
        if gpus:
            for gpu in gpus:
                if not tf.config.experimental.get_memory_growth(gpu):
                    tf.config.experimental.set_memory_growth(gpu, True)
            print(f"✅ GPU (RTX 4050) für Heatmap-Erstellung bereit.")
    except RuntimeError:
        print("ℹ️ GPU war bereits initialisiert. Fahre fort...")

setup_gpu()

# --- 2. Konfiguration ---
class HeatmapConfig:
    model_path = 'final_model.keras'
    base_path = Path('test_patches')
    patch_folder = base_path / "patches"
    metadata_file = base_path / "metadata.txt"
    output_folder = Path('heatmaps_output')
    
    batch_size = 32  # Passt gut in den 6GB VRAM der RTX 4050

# --- 3. Hauptfunktion ---
def generate_gpu_heatmaps():
    cfg = HeatmapConfig()
    cfg.output_folder.mkdir(exist_ok=True)

    if not cfg.metadata_file.exists():
        print(f"❌ {cfg.metadata_file} nicht gefunden!")
        return

    # Modell laden
    print("⏳ Lade Modell...")
    model = load_model(cfg.model_path)

    # Metadaten einlesen
    with open(cfg.metadata_file, "r") as f:
        data_lines = [line.strip().split(";") for line in f.readlines()]

    # Gruppierung nach Originalbild
    images_dict = {}
    for img_name, rel_patch_path, px, py, ps, w_orig, h_orig in data_lines:
        if img_name not in images_dict:
            images_dict[img_name] = {"w": int(w_orig), "h": int(h_orig), "patches": []}
        images_dict[img_name]["patches"].append({
            "rel_path": rel_patch_path, "x": int(px), "y": int(py), "s": int(ps)
        })

    print(f"--- Starte Heatmap-Generierung für {len(images_dict)} Bilder ---")

    for img_name, info in images_dict.items():
        h, w = info["h"], info["w"]
        # Initialisierung mit Nullen
        heatmap_max = np.zeros((h, w), dtype=np.float32)

        print(f"🚀 Verarbeite: {img_name}")

        # --- BATCH LOADING ---
        patch_list = info["patches"]
        all_patch_imgs = []
        
        for p in patch_list:
            # Schnelles Laden via OpenCV (Graustufen, 0-255 Scale)
            p_img = cv2.imread(str(cfg.patch_folder / p["rel_path"]), cv2.IMREAD_GRAYSCALE)
            if p_img is None: continue
            p_img = cv2.resize(p_img, (256, 256))
            all_patch_imgs.append(p_img.astype(np.float32))

        if not all_patch_imgs:
            continue

        # In Tensor-Format umwandeln (Batch, Height, Width, Channels)
        input_batch = np.expand_dims(np.array(all_patch_imgs), axis=-1)

        # --- GPU INFERENCE ---
        # Alle Patches des Bildes in einem Rutsch vorhersagen
        preds = model.predict(input_batch, batch_size=cfg.batch_size, verbose=0)

        # --- REKONSTRUKTION ---
        for i, prob_vec in enumerate(preds):
            prob = float(prob_vec[0]) if len(prob_vec) == 1 else float(prob_vec[1])
            
            p = patch_list[i]
            px, py, ps = p["x"], p["y"], p["s"]
            
            # Maximum-Logik (Pfad A): Der höchste Wert pro Pixel gewinnt
            # Das verhindert das "Verwässern" durch schlechte Patches
            heatmap_max[py:py+ps, px:px+ps] = np.maximum(heatmap_max[py:py+ps, px:px+ps], prob)

        # --- SPEICHERN ---
        # Umwandlung in 8-Bit Grayscale
        heatmap_8bit = (heatmap_max * 255).astype(np.uint8)
        
        # Optional: Falls du eine farbige Heatmap willst, aktiviere die nächste Zeile:
        # heatmap_8bit = cv2.applyColorMap(heatmap_8bit, cv2.COLORMAP_JET)

        output_path = cfg.output_folder / f"{Path(img_name).stem}_heatmap.png"
        cv2.imwrite(str(output_path), heatmap_8bit)

    print(f"✅ Alle Heatmaps wurden in '{cfg.output_folder}' erstellt.")

# --- Startbefehl ---
if __name__ == "__main__":
    generate_gpu_heatmaps()

ℹ️ GPU war bereits initialisiert. Fahre fort...
⏳ Lade Modell...
--- Starte Heatmap-Generierung für 200 Bilder ---
🚀 Verarbeite: Passat_P3_frame3520.jpg
🚀 Verarbeite: Passat_P3_frame460.jpg
🚀 Verarbeite: Focus_P2_frame_1030.jpg
🚀 Verarbeite: Focus_P1_frame_1430.jpg
🚀 Verarbeite: Passat_P3_frame380.jpg
🚀 Verarbeite: Seat_N_47.jpg
🚀 Verarbeite: Passat_P1_frame570.jpg
🚀 Verarbeite: Passat_N3_frame2120.jpg
🚀 Verarbeite: Passat_P3_frame2910.jpg
🚀 Verarbeite: CKlasse_Bilderreihe 5_P_IMG_9514_rot-3.jpg
🚀 Verarbeite: Focus_P1_frame_930.jpg
🚀 Verarbeite: Focus_P1_frame_3540.jpg
🚀 Verarbeite: Focus_P1_frame_730.jpg
🚀 Verarbeite: Focus_P1_frame_3220.jpg
🚀 Verarbeite: Passat_N1_frame3050.jpg
🚀 Verarbeite: GKLASSE_NEU_P1_frame1980.jpg
🚀 Verarbeite: Passat_P2_frame3090.jpg
🚀 Verarbeite: Golf_IV_P_43.jpg
🚀 Verarbeite: IMG_6903.jpg
🚀 Verarbeite: 20251124_142412.jpg
🚀 Verarbeite: Passat_N2_frame960.jpg
🚀 Verarbeite: Passat_P3_frame630.jpg
🚀 Verarbeite: Passat_P2_frame950.jpg
🚀 Verarbeite: CKlasse_Bilde

### Vergleich der erzeugten Ergebnisse

In [54]:
import cv2
import numpy as np
from pathlib import Path

# --- 1. Konfiguration der Pfade ---
path_original = Path('test_picture')
path_heatmap = Path('heatmaps_output')
path_decision = Path('final_results')

# --- 2. FIXE FENSTERGRÖSSE ---
WINDOW_WIDTH = 1500
WINDOW_HEIGHT = 500

def show_interactive_evaluation():
    # Bilder suchen
    img_files = sorted(list(path_original.glob("*.[jJ][pP][gG]")) + list(path_original.glob("*.[pP][nN][gG]")))
    
    if not img_files:
        print("❌ Keine Originalbilder gefunden.")
        return

    single_w = WINDOW_WIDTH // 3
    single_h = WINDOW_HEIGHT

    # --- COUNTER INITIALISIEREN ---
    stats = {
        "richtig": 0,
        "falsch": 0,
        "gesamt": 0
    }

    print(f"--- Interaktive Evaluation gestartet ---")
    print("Steuerung: [J] = Richtig erkannt | [N] = Falsch erkannt | [Q] = Beenden")

    for idx, img_file in enumerate(img_files):
        stem = img_file.stem
        
        # 1. Bilder laden
        img_orig = cv2.imread(str(img_file))
        
        # Heatmap suchen (verschiedene Endungen)
        heatmap_path = next((p for p in [path_heatmap / f"{stem}_heatmap.png", path_heatmap / f"{stem}_heatmap.jpg"] if p.exists()), None)
        img_heat = cv2.imread(str(heatmap_path)) if heatmap_path else None
        
        # Ergebnis suchen
        final_path = next((p for p in [path_decision / f"final_{img_file.name}", path_decision / f"result_{img_file.name}"] if p.exists()), None)
        img_final = cv2.imread(str(final_path)) if final_path else None

        # Falls ein Teil fehlt, überspringen
        if img_orig is None or img_heat is None or img_final is None:
            continue

        # 2. Resizing
        res_orig = cv2.resize(img_orig, (single_w, single_h), interpolation=cv2.INTER_AREA)
        res_heat = cv2.resize(img_heat, (single_w, single_h), interpolation=cv2.INTER_AREA)
        res_final = cv2.resize(img_final, (single_w, single_h), interpolation=cv2.INTER_AREA)

        # 3. Farbraum-Check
        if len(res_heat.shape) == 2:
            res_heat = cv2.cvtColor(res_heat, cv2.COLOR_GRAY2BGR)

        # 4. Horizontal stapeln
        triple_view = np.hstack((res_orig, res_heat, res_final))

        # 5. Beschriftungen (Header)
        header_text = [("ORIGINAL", 10), ("HEATMAP", single_w + 10), ("DETEKTION", (single_w * 2) + 10)]
        for text, pos_x in header_text:
            cv2.putText(triple_view, text, (pos_x + 2, 32), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 0), 2)
            cv2.putText(triple_view, text, (pos_x, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

        # 6. Status-Leiste (Footer)
        # Balken unten für bessere Lesbarkeit
        cv2.rectangle(triple_view, (0, single_h - 40), (WINDOW_WIDTH, single_h), (50, 50, 50), -1)
        
        status_str = f"Bild {idx+1}/{len(img_files)} | Richtig: {stats['richtig']} | Falsch: {stats['falsch']} | DRUECKE: [J]a / [N]ein"
        cv2.putText(triple_view, status_str, (20, single_h - 12), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        # 7. Anzeige & Interaktions-Loop
        cv2.imshow("QR-Analyse Evaluation", triple_view)
        
        valid_key = False
        while not valid_key:
            key = cv2.waitKey(0) & 0xFF
            
            if key == ord('q'):
                print("\n🛑 Abbruch durch Benutzer.")
                print_summary(stats)
                cv2.destroyAllWindows()
                return

            elif key == ord('j'): # Ja / Richtig
                stats["richtig"] += 1
                stats["gesamt"] += 1
                print(f"✅ Bild {idx+1}: Richtig markiert.")
                valid_key = True

            elif key == ord('n'): # Nein / Falsch
                stats["falsch"] += 1
                stats["gesamt"] += 1
                print(f"❌ Bild {idx+1}: Falsch markiert.")
                valid_key = True
            
            else:
                print("⚠️  Bitte 'j' für Richtig, 'n' für Falsch oder 'q' zum Beenden drücken.")

    print_summary(stats)
    cv2.destroyAllWindows()

def print_summary(stats):
    total = stats["gesamt"]
    if total == 0:
        print("Keine Bilder bewertet.")
        return
        
    acc = (stats["richtig"] / total) * 100
    print("\n" + "="*40)
    print(f"📊 ABSCHLUSSBERICHT")
    print("="*40)
    print(f"Gesamt bewertet: {total}")
    print(f"✅ Richtig:       {stats['richtig']}")
    print(f"❌ Falsch:        {stats['falsch']}")
    print("-" * 40)
    print(f"📈 Genauigkeit:   {acc:.2f}%")
    print("="*40)

if __name__ == "__main__":
    show_interactive_evaluation()

--- Interaktive Evaluation gestartet ---
Steuerung: [J] = Richtig erkannt | [N] = Falsch erkannt | [Q] = Beenden
✅ Bild 1: Richtig markiert.
✅ Bild 2: Richtig markiert.
✅ Bild 3: Richtig markiert.
✅ Bild 4: Richtig markiert.
✅ Bild 5: Richtig markiert.
✅ Bild 6: Richtig markiert.
✅ Bild 7: Richtig markiert.
✅ Bild 8: Richtig markiert.
✅ Bild 9: Richtig markiert.
✅ Bild 10: Richtig markiert.
✅ Bild 11: Richtig markiert.
✅ Bild 12: Richtig markiert.
✅ Bild 13: Richtig markiert.
✅ Bild 14: Richtig markiert.
✅ Bild 15: Richtig markiert.
✅ Bild 16: Richtig markiert.
✅ Bild 17: Richtig markiert.
✅ Bild 18: Richtig markiert.
✅ Bild 19: Richtig markiert.
✅ Bild 20: Richtig markiert.
✅ Bild 21: Richtig markiert.
✅ Bild 22: Richtig markiert.
✅ Bild 23: Richtig markiert.
✅ Bild 24: Richtig markiert.
✅ Bild 25: Richtig markiert.
✅ Bild 26: Richtig markiert.
✅ Bild 27: Richtig markiert.
✅ Bild 28: Richtig markiert.
✅ Bild 29: Richtig markiert.
✅ Bild 30: Richtig markiert.
✅ Bild 31: Richtig markier

### Alte Patches, Bilder, Heatmaps und Results löschen